In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

train = pd.read_csv('../input/Kannada-MNIST/train.csv')
dig_mnist = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')


X_train = train.iloc[:,1:].to_numpy(dtype=np.float32, copy=True)
y_train = train.iloc[:,0].to_numpy()
y_train = to_categorical(y_train, num_classes=10)
X_train = X_train.reshape(-1, 28,28,1)
X_train = X_train / 255

X_test = dig_mnist.iloc[:,1:].to_numpy(dtype=np.float32, copy=True)
y_test = dig_mnist.iloc[:, 0].to_numpy()
y_test = to_categorical(y_test, num_classes=10)

X_test = X_test.reshape(-1,28,28,1)
X_test = X_test / 255

X_train, X_val, y_train, y_val = train_test_split(np.concatenate((X_train, X_test)),
                                                 np.concatenate((y_train, y_test)),
                                                 test_size=0.1, random_state=1,shuffle=True)


one_image = X_train[46]

plt.imshow(one_image.reshape(28,28), cmap='gray')
print(f'This is an image of {y_train[46]}')
plt.show()


In [ ]:
'''
#baseline model
base_model = Sequential()
base_model.add(Input(shape=(28,28,1)))
base_model.add(Dense(10, activation='softmax'))

base_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='sgd')
history = base_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
'''

In [ ]:
'''
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(train_acc))

plt.plot(epochs, train_acc,'o-g', label='Train')
plt.plot(epochs, val_acc,'o-b', label='Val')
plt.legend()
plt.show()

plt.plot(epochs, train_loss, 'o-g', label='Train')
plt.plot(epochs, val_loss, 'o-b', label='Val')
plt.legend()
plt.show()
'''

In [ ]:
'''
#Simple CNN

#reshape to image dim
X_train = X_train.reshape(-1, 28, 28, 1)
print(X_train.shape)
X_train.dtype

simple_model = Sequential()
simple_model.add(Input(shape=(28,28,1)))
simple_model.add(Conv2D(32, 5, padding='same', activation='relu')) 
simple_model.add(MaxPool2D())
simple_model.add(Conv2D(64, 5, padding='same', activation='relu'))
simple_model.add(MaxPool2D())
simple_model.add(Flatten())
simple_model.add(Dense(100, activation='relu'))
simple_model.add(Dense(10, activation='softmax'))
                 
simple_model.compile(loss='categorical_crossentropy', optimizer='adam',
                    metrics=['accuracy'])
                 
history = simple_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
'''

In [ ]:
'''
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(train_acc))

plt.plot(epochs, train_acc,'o-g', label='Train')
plt.plot(epochs, val_acc,'o-b', label='Val')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, train_loss, 'o-g', label='Train')
plt.plot(epochs, val_loss, 'o-b', label='Val')
plt.title('Loss')
plt.legend()
plt.show()
'''

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
#build better model

def cnn_model():
    inp = tf.keras.Input(shape=(28,28,1))
    x1 = tf.keras.layers.Conv2D(128, (1,1), strides=(1,1), activation='relu')(inp)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x3 = tf.keras.layers.Conv2D(128, (3,3), padding='same', strides=(1,1), activation='relu')(inp)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x5 = tf.keras.layers.Conv2D(128, (5,5), padding='same', strides=(1,1), activation='relu')(inp)
    x5 = tf.keras.layers.BatchNormalization()(x5)
    
    #take average of each set (x1, x3, x5)
    averaged = tf.keras.layers.Average()([x1,x3,x5])
    averaged = tf.keras.layers.Activation('relu')(averaged)
    averaged = tf.keras.layers.BatchNormalization()(averaged)

    #apply to average values
    x = tf.keras.layers.Conv2D(128, (5,5), strides=(1,1), activation='relu')(averaged)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.25)(x)

    #second layer
    x1 = tf.keras.layers.Conv2D(128, (1,1), strides=(1,1), activation='relu')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x3 = tf.keras.layers.Conv2D(128, (3,3), padding='same', strides=(1,1), activation='relu')(x)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x5 = tf.keras.layers.Conv2D(128, (5,5), padding='same', strides=(1,1), activation='relu')(x)
    x5 = tf.keras.layers.BatchNormalization()(x5)

    averaged = tf.keras.layers.Average()([x1,x3,x5])
    averaged = tf.keras.layers.Activation('relu')(averaged)
    averaged = tf.keras.layers.BatchNormalization()(averaged)

    x = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), activation='relu')(averaged)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.25)(x)

    #third layer
    x1 = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), activation='relu')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x3 = tf.keras.layers.Conv2D(256, (3,3), padding='same', strides=(1,1), activation='relu')(x)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x5 = tf.keras.layers.Conv2D(256, (5,5), padding='same', strides=(1,1), activation='relu')(x)
    x5 = tf.keras.layers.BatchNormalization()(x5)

    averaged = tf.keras.layers.Average()([x1,x3,x5])
    averaged = tf.keras.layers.Activation('relu')(averaged)
    averaged = tf.keras.layers.BatchNormalization()(averaged)

    x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), activation='relu')(averaged)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.25)(x)

    #fourth layer
    x1 = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), activation='relu')(x)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x3 = tf.keras.layers.Conv2D(256, (3,3), padding='same', strides=(1,1), activation='relu')(x)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x5 = tf.keras.layers.Conv2D(256, (5,5), padding='same', strides=(1,1), activation='relu')(x)
    x5 = tf.keras.layers.BatchNormalization()(x5)

    averaged = tf.keras.layers.Average()([x1,x3,x5])
    averaged = tf.keras.layers.Activation('relu')(averaged)
    averaged = tf.keras.layers.BatchNormalization()(averaged)

    x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), activation='relu')(averaged)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.25)(x)
    
    #cnn
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation=tf.nn.relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(128, activation=tf.nn.relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    
    output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(x)

    model = tf.keras.Model(inputs=inp, outputs=output)
    return model

BATCH_SIZE= 128
model=cnn_model()
opt = Adam(learning_rate = 0.001)
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.75)




In [ ]:
#alter train data to produce more data
train_aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                                                  width_shift_range=0.2,
                                                  height_shift_range=0.2,
                                                  shear_range=0.1,
                                                  zoom_range=0.2,
                                                  horizontal_flip=False)
valid_aug = tf.keras.preprocessing.image.ImageDataGenerator()

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, callbacks=[lr_reduction, es], verbose=1,
         validation_data=(X_val, y_val))

model.fit_generator(train_aug.flow(X_train, y_train,batch_size=BATCH_SIZE),
                    steps_per_epoch=10,
                    validation_data=valid_aug.flow(X_val, y_val),
                    validation_steps=50,
                    epochs=5, verbose=1,
                       callbacks=[lr_reduction, es])

In [ ]:
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(train_acc))

plt.plot(epochs, train_acc,'o-g', label='Train')
plt.plot(epochs, val_acc,'o-b', label='Val')
plt.title('Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, train_loss, 'o-g', label='Train')
plt.plot(epochs, val_loss, 'o-b', label='Val')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
#make predictions on real data
real_test = pd.read_csv('../input/Kannada-MNIST/test.csv')

real_test = real_test.iloc[:,1:].to_numpy()
real_test = real_test / 255
real_test = real_test.reshape(-1, 28, 28, 1)

sample_submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')

predictions = model.predict(real_test)
predictions = predictions.argmax(axis=1)

predictions = predictions.astype(int).flatten()
predictions = (LabelEncoder().fit_transform((predictions)))
sample_submission['Label'] = predictions
sample_submission.to_csv('Submission file', index=False)